In [11]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Add, Reshape,Conv2D,MaxPooling2D,ZeroPadding2D,experimental,AveragePooling2D,Concatenate,Dropout,Activation
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import numpy as np
import keras
from keras import layers
from PIL import Image
import cv2
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import accuracy_score
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
import os
import numpy as np
#import cPickle
import _pickle as cPickle
import pandas as pd 
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential 
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '../input/dermnet/train',
    target_size=(300, 300),
    batch_size=8,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '../input/dermnet/test',
    target_size=(300, 300),
    batch_size=8,
    class_mode='categorical')

Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [14]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(300,300,3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(23, activation='softmax'))

In [15]:
batch_size = 16
checkpoint_filepath = 'DERM_DNN'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            monitor='val_accuracy',mode='max',
                                            save_best_only=True)
opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
history = model.fit_generator(train_generator,epochs=100,verbose=2, callbacks=[model_checkpoint_callback])

Epoch 1/100
1945/1945 - 118s - loss: 28.4532 - accuracy: 0.1032
Epoch 2/100
1945/1945 - 119s - loss: 2.9619 - accuracy: 0.1650
Epoch 3/100
1945/1945 - 118s - loss: 2.7879 - accuracy: 0.2106
Epoch 4/100
1945/1945 - 118s - loss: 2.5622 - accuracy: 0.2676
Epoch 5/100
1945/1945 - 118s - loss: 2.3310 - accuracy: 0.3335
Epoch 6/100
1945/1945 - 118s - loss: 2.0710 - accuracy: 0.4030
Epoch 7/100
1945/1945 - 121s - loss: 1.8241 - accuracy: 0.4782
Epoch 8/100
1945/1945 - 119s - loss: 1.6067 - accuracy: 0.5400
Epoch 9/100
1945/1945 - 119s - loss: 1.4086 - accuracy: 0.6021
Epoch 10/100
1945/1945 - 119s - loss: 1.2478 - accuracy: 0.6451
Epoch 11/100
1945/1945 - 121s - loss: 1.1297 - accuracy: 0.6810
Epoch 12/100
1945/1945 - 120s - loss: 1.0155 - accuracy: 0.7203
Epoch 13/100
1945/1945 - 120s - loss: 0.9318 - accuracy: 0.7465
Epoch 14/100
1945/1945 - 120s - loss: 0.8656 - accuracy: 0.7643
Epoch 15/100
1945/1945 - 121s - loss: 0.7872 - accuracy: 0.7930
Epoch 16/100
1945/1945 - 120s - loss: 0.7479 - a

In [ ]:
model.evaluate(validation_generator)

In [16]:
def pre_trained_model():
    input = Input((300,300,3))
    c1l1  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(input)
    c1l2  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l1)
    ces2  = Add()([c1l2, c1l1])
    c1l3  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(ces2)
    ces3  = Add()([c1l3, ces2])
    ces3  = Flatten()(ces3)
    ces3  = Dense(23)(ces3)
    ces3  = Activation('softmax')(ces3)
    return Model(inputs=input, outputs=ces3)

In [17]:
pretrained_model = pre_trained_model()
pretrained_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
checkpoint_filepath = 'DERM_shallowNN'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               monitor='val_accuracy',mode='max',
                                                               save_best_only=True)
history = pretrained_model.fit_generator(train_generator,epochs = 100,callbacks = [model_checkpoint_callback])

Epoch 1/100
1945/1945 [==============================] - 205s 105ms/step - loss: 7.3386 - accuracy: 0.1552
Epoch 2/100
1945/1945 [==============================] - 203s 104ms/step - loss: 2.7087 - accuracy: 0.2553
Epoch 3/100
1945/1945 [==============================] - 203s 104ms/step - loss: 2.0708 - accuracy: 0.4494
Epoch 4/100
1945/1945 [==============================] - 203s 104ms/step - loss: 1.3408 - accuracy: 0.6916
Epoch 5/100
1945/1945 [==============================] - 204s 105ms/step - loss: 0.8860 - accuracy: 0.8245
Epoch 6/100
1945/1945 [==============================] - 204s 105ms/step - loss: 0.6807 - accuracy: 0.8880
Epoch 7/100
1945/1945 [==============================] - 203s 105ms/step - loss: 0.5781 - accuracy: 0.9143
Epoch 8/100
1945/1945 [==============================] - 203s 105ms/step - loss: 0.4929 - accuracy: 0.9317
Epoch 9/100
1945/1945 [==============================] - 204s 105ms/step - loss: 0.4837 - accuracy: 0.9395
Epoch 10/100
1945/1945 [=============

KeyboardInterrupt: 

In [ ]:
pretrained_model.evaluate(validation_generator)

In [ ]:
def knowledge_incorporated_model(pretrained_model):    
    input = Input((32,32,3))
    inputB = Input((10))
    l1    = pretrained_model(input)
    l1    = Dense(10)(l1)
    l2    = Activation('sigmoid')(l1)
    l3    = Add()([l2,inputB])
    l4    = Dense(128)(l3)
    l5    = Activation('relu')(l4)
    l6    = Dense(10)(l5)
    l7    = Activation('sigmoid')(l6)
    l8    = Add()([l7,inputB])
    l9    = Activation('softmax')(l8)
    return Model(inputs=[input, inputB], outputs=l9)

In [ ]:
inp = Input((32,32,3))
l1  = pre_trained_model.layers[1](inp)
l2  = pre_trained_model.layers[2](l1)
l3  = pre_trained_model.layers[3]([l1,l2])
l4  = pre_trained_model.layers[4](l3)
x   = pre_trained_model.layers[5]([l3,l4])
x   = pre_trained_model.layers[6](x)
model = Model(inp,x)

In [ ]:
model_DNN_knowledge = knowledge_incorporated_model(model)
model_DNN_knowledge.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_filepath = 'model_DNN_knowledge_cifar'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               monitor='val_accuracy',mode='max',
                                                               save_best_only=True)
history = model_DNN_knowledge.fit(x = [x_train,knowledge],y = y_train,epochs = 150,callbacks = [model_checkpoint_callback],
                     validation_split=0.1)

In [19]:
tf.keras.models.save_model(model,'./dermDNN')
tf.keras.models.save_model(pretrained_model,'./dermDNN_shallow')


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
